Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [2]:
from random import choices

import lab9_lib

In [3]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

00011111110101000111000001000010010111100111110101: 23.34%
11001100101101111010011001101110110100001101001100: 23.33%
10100011100100101100101010111001110001111011001000: 23.34%
01011100001010010101100101010110000100100010101011: 7.34%
11011000001010000010110011111011111110010100000100: 15.36%
11001001011001001011110101111010011001011100101001: 9.13%
11101100011011110011100000100000111010110101011101: 7.33%
11111000011011001011010110010000011111010010100111: 23.34%
01101000101111100011001110110000101100010010110010: 19.36%
11111110111010000000010011100110010010110101100110: 41.56%
10


In [4]:
import numpy as np
import numpy.typing as npt
import networkx as nx
from typing import List, Tuple, Optional, Callable, Union
from itertools import chain, combinations
from functools import reduce
from tqdm import tqdm

class Agent:

    def __init__(self, genome: Optional[np.array] = None, num_loci: int = 1000) -> None:
        """
        num_rows: number of rows of the game
        genome: initial parameters
        k: largest number of piles it can take
        """
        if genome is None:
            self._genome = np.random.choice([0,1], num_loci)
        else:
            self._genome = genome
        
        self.fitness = 0
        self.explicit_fitness = False
    
    def __lt__(self, other: 'Agent'):
        return self._fitness < other._fitness

    def __hash__(self):
        return hash(self._genome)

    def __eq__(self, other):
        if isinstance(other, Agent):
            return self._genome == other._genome
        return False

    @property
    def genome(self):
        return self._genome

    @genome.setter
    def genome(self, p):
        self._genome = p
    
    @property
    def fitness(self):
        return self._fitness

    @fitness.setter
    def fitness(self, fitness: float):
        self._fitness = fitness
        self.explicit_fitness = True
    
    def set_implicit_fitness(self, fitness: float):
        self._fitness = fitness
        self.explicit_fitness = False
    
    def reset(self):
        self.fitness = 0
    
    def __iadd__(self, other) -> None:
        self._genome += other


class EvolutionTask:

    def __init__(self, fitness: 'Problem') -> None:
        self.is_island = False
        self.fitness = fitness
        self.best_fitness = 0
    
    def mutate(self, agent: Agent, p: float = 0.001) -> Agent:
        """
        Mutates loci with random flipping and returns a new agent
        """
        new_genome = []
        for locus in agent.genome:
            if np.random.rand()<p:
                new_genome.append(1-locus)
            else:
                new_genome.append(locus)
        return Agent(np.array(new_genome))
    
    def compute_similarity(self, a: Agent, b: Agent):
        sim = [a.genome[i]==b.genome[i] for i in range(len(a.genome))]
        return sum(sim)/len(sim)

    def compute_similarity_matrix(self) -> None:
        if self.similarity_matrix is None:
            G = nx.Graph()
        else:
            G = self.similarity_matrix
        for a in self.agents:
            for b in self.agents:
                if a not in G or b not in G:
                    G.add_edge(a,b, weight=self.compute_similarity(a,b))
        self.similarity_matrix = G

    def calculate_fitness(self, explicit: bool=True, sample_size: Union[int, float] = 0) -> None:
        """
        Calculates the fitness of the current population
        """
        if explicit:
            for agent in self.agents:
                agent.fitness = self.fitness(agent.genome)
                if agent.fitness > self.best_fitness:
                    self.best_fitness = agent.fitness
        else:
            self.compute_similarity_matrix()
            stubborn_agents_indeces = np.random.choice(range(len(self.agents)), size=sample_size, replace=False)
            u = np.zeros_like(self.agents)
            u[stubborn_agents_indeces] = [a.fitness for a in self.agents[stubborn_agents_indeces]]
            L = nx.laplacian_matrix(self.similarity_matrix).to_array()
            for i in stubborn_agents_indeces:
                L[i,:] = np.zeros_like(self.agents)
                L[i,i] = self.agents[i].fitness
            new_fitness = np.linalg.inv(L)*u
            for i, f in enumerate(new_fitness):
                if i not in stubborn_agents_indeces:
                    self.agents[i].set_implicit_fitness(f)
    
    def crossover(self, a1: Agent, a2: Agent, a: float=1, b: float=999) -> Agent:
        """
        Given two agents it randomly selects the parameters between the two
        """
        n = int(np.random.beta(a,b)*len(a1.genome))
        new_genome = np.concatenate((a1.genome[0:n], a2.genome[n:]))
        return Agent(new_genome)
    
    def es_iter(self, agents: Optional['np.array']=None, population_size: int=100, mu: int=30, strategy='comma', p=0.001) -> None:
        
        if agents is None:
            self.agents = np.partition(self.agents, population_size-mu)[population_size-mu:] # selective pressure, takes top mu agents
        else:
            self.agents = agents

        self.calculate_fitness()

        if strategy == 'comma':
            num_children = population_size
        else:
            num_children = population_size-mu
        
        parents = np.random.choice(self.agents, num_children)
        children = [self.mutate(a, p) for a in parents]
        if strategy == 'comma':
            self.agents = children
        else:
            self.agents = children + list(self.agents)
    
    def ga_iter(self, agents: Optional['np.array']=None, population_size: int=100, mu: int=30, strategy='comma', p=0.001, a_beta=1, b_beta=999) -> None:
        
        if agents is None:
            self.agents = np.partition(self.agents, population_size-mu)[population_size-mu:] # selective pressure, takes top mu agents
        else:
            self.agents = agents

        self.calculate_fitness()

        if strategy == 'comma':
            num_children = population_size
        else:
            num_children = population_size-mu
        
        parents = np.random.choice(self.agents, num_children)
        children = [self.crossover(a[0], a[1], a_beta, b_beta) for a in np.random.choice(parents, (num_children, 2))]
        children = [self.mutate(a, p) for a in children]
        if strategy == 'comma':
            self.agents = children
        else:
            self.agents = children + list(self.agents)
    
    def es(self, n_generations: int=100, population_size: int=100, mu: int=30, strategy='comma', p=0.001) -> None:
        self.is_island = False
        self.agents: 'np.array' = np.array([Agent() for _ in range(population_size)])
        for _ in tqdm(range(n_generations)):
            self.es_iter(population_size=population_size, mu=mu, strategy=strategy, p=p)
        self.calculate_fitness()
    
    def genetic_algorithm(self, n_generations: int=100, population_size: int=100, mu: int=30, strategy='comma', p=0.001, a_beta=1, b_beta=999) -> None:
        self.is_island = False
        self.agents: 'np.array' = np.array([Agent() for _ in range(population_size)])
        for _ in tqdm(range(n_generations)):
            self.ga_iter(population_size=population_size, mu=mu, strategy=strategy, p=p, a_beta=a_beta, b_beta=b_beta)
        self.calculate_fitness()
    
    def migrate(self, migration_size: int=1) -> 'np.array':
        return np.random.choice(self.agents, migration_size)
    
    def receive_migrants(self, migrants: 'np.array'):
        self.agents = np.concatenate((self.agents, migrants))
    
    def island_model(self, n_generations: int=100, migration_period: int=1, migration_size: int=1, island_size: int=10, n_islands: int = 10, mu_island: int=3, strategy='comma', p=0.001, a_beta=1, b_beta=999, replace=False):
        self.is_island = True
        self.islands: 'np.array' = [EvolutionTask(self.fitness) for _ in range(n_islands)]
        for i in tqdm(range(n_generations)):
            for island in self.islands:
                if i == 0:
                    island.ga_iter(agents=np.array([Agent() for _ in range(island_size)]), population_size=island_size, mu=mu_island, strategy=strategy, p=p, a_beta=a_beta, b_beta=b_beta)
                else:
                    island.ga_iter(population_size=island_size, mu=mu_island, strategy=strategy, p=p, a_beta=a_beta, b_beta=b_beta)
            if i%migration_period==0:
                migrants = np.concatenate([island.migrate(migration_size) for island in self.islands])
                migrants = np.random.choice(migrants, size=(n_islands, migration_size), replace=replace)
                for j, island in enumerate(self.islands):
                    island.receive_migrants(migrants[j,:])
            for island in self.islands:
                island.calculate_fitness()
        for island in self.islands:
            island.calculate_fitness()
    
    @property
    def best_fitness(self) -> float:
        if self.is_island:
            return max([island.best_fitness for island in self.islands])
        return self.best_fitness
    
    @property
    def fitness_calls(self) -> int:
        return self.fitness.calls

In [5]:
fitness = lab9_lib.make_problem(10)
et = EvolutionTask(fitness)
et.es(n_generations=10, mu=50)
print(et.best_fitness, fitness.calls)


100%|██████████| 10/10 [00:00<00:00, 12.47it/s]

0.11360234638 600


In [6]:
fitness = lab9_lib.make_problem(10)
et = EvolutionTask(fitness)
et.genetic_algorithm(n_generations=10, mu=50)
print(et.best_fitness, fitness.calls)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 10.65it/s]

0.11000135589 600


In [11]:
fitness = lab9_lib.make_problem(5)
et = EvolutionTask(fitness)
et.island_model(n_generations=1000, mu_island=30, migration_size=20, island_size=100)
print(et.best_fitness, fitness.calls)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [15:16<00:00,  1.09it/s]

0.43072000000000005 1701700
